# dimensionality reduction
In this notebook, we will explore the dimensionality reduction techniques. We will use a sinmple dataset of 1000 samples with 100 features. We will use different dimensionality reduction techniques to reduce the dimensionality of the data and visualize the data in 2D space.
Here is the list of dimensionality reduction techniques we will explore in this notebook:
1. Principal Component Analysis (PCA)
2. t-distributed Stochastic Neighbor Embedding (tSNE)
3. Independent Component Analysis (ICA)
4. Spectral Embedding
5. Linear Discriminant Analysis (LDA)
6. Isomap
7. Locally Linear Embedding (LLE)
8. Multidimensional Scaling (MDS)

These methods have advantages and disadvantages. For example, PCA is very fast but it loses the finer details of the data after the reduction. On the other hand, tSNE is very slow but it preserves the underlying structure of the data.


# PCA 
PCA preserves the Maximum Variance in the data. The pricipal components are the directions of the data with the maximum variance. The first principal component is the direction of the data with the maximum variance. Mathematically speaking, we are solving the following optimization problem:
$$
\underset{w}{\text{maximize}} \frac{1}{n} \sum_{i=1}^{n} (w^T x_i)^2
$$
where $w$ is the direction of the data with the maximum variance. The solution to this optimization problem is the first principal component. The second principal component is the direction of the data with the maximum variance that is orthogonal to the first principal component. Mathematically speaking, we are solving the following optimization problem:
$$
\underset{w}{\text{maximize}} \frac{1}{n} \sum_{i=1}^{n} (w^T x_i)^2
$$
subject to the constraint that $w$ is orthogonal to the first principal component. The constraint can be written as $w^T w_1 = 0$.

## PCA in python from scratch
We can implement PCA in python from scratch. Here is the code:
```python
def pca_vanilla(X, n_components):
    # center the data
    X = X - np.mean(X, axis=0)
    # calculate the covariance matrix
    cov = np.cov(X.T)
    # calculate the eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(cov)
    # sort the eigenvalues and eigenvectors in descending order
    idx = eigenvalues.argsort()[::-1]   
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    # select the first n_components eigenvectors
    eigenvectors = eigenvectors[:, :n_components]
    # project the data onto the first n_components eigenvectors
    X_transformed = np.dot(X, eigenvectors)
    return X_transformed
```
### PCA using singular value decomposition (SVD)
We can also implement PCA using singular value decomposition (SVD). Here is the code:
```python
def pca_svd(X, n_components):
    # center the data
    X = X - np.mean(X, axis=0)
    # calculate the covariance matrix
    cov = np.cov(X.T)
    # calculate the eigenvalues and eigenvectors of the covariance matrix
    U, S, V = np.linalg.svd(cov)
    # select the first n_components eigenvectors
    U = U[:, :n_components]
    # project the data onto the first n_components eigenvectors
    X_transformed = np.dot(X, U)
    return X_transformed
```

*** What does maximum variance in first principal component mean? ***
It means that the data points are spread out as much as possible along the first principal component. 
Why is this important? Because the first principal component captures the maximum amount of information about the data. 

*** What does maximum variance in second principal component mean? ***
It means that the data points are spread out as much as possible along the second principal component. 

*** Why do we need to preserve the maximum variance in the data? ***
Because the data points are spread out as much as possible along the components. This means that the data are as informative as possible.

*** How do we find which features are building the principal component? ***
How to find features that are contributing the most to the variance of the data? How to find features that are contributing the most to the variance along the first principal component? This can be done by calculating the correlation between the features and the first principal component. The features with the highest correlation are the features that are contributing the most to the variance along the first principal component. In python we can use the following code to find the features that are contributing the most to the variance along the first principal component.
```python
def find_features_contributing_most_to_variance_along_first_principal_component(X):
    pca = PCA(n_components=1)
    pca.fit(X)
    first_principal_component = pca.components_[0]
    correlation = np.corrcoef(X.T, first_principal_component)
    return np.argsort(correlation[0, :-1])[::-1]
```


In [8]:
# Create a regression data set with 1000 samples and 10 features and 5 informative features

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn import manifold
from sklearn import decomposition
from sklearn import random_projection



In [41]:
from sklearn import datasets

from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
X = iris.data
y = iris.target
# Z-score the features
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


In [42]:
def pca_svd(X, n_components, return_components=False):
    # center the data
    X = X - np.mean(X, axis=0)
    # calculate the covariance matrix
    cov = np.cov(X.T)
    # calculate the eigenvalues and eigenvectors of the covariance matrix
    U, S, V = np.linalg.svd(cov)
    # print the explained variance
    explained_variance = np.cumsum(S**2) / np.sum(S**2)
    print("Explained variance: ", explained_variance)
    # select the first n_components eigenvectors
    U = U[:, :n_components]
    # project the data onto the first n_components eigenvectors
    X_transformed = np.dot(X, U)
    if return_components:
        return X_transformed, U
    return X_transformed

def find_features_contributing_most_to_variance_to_components(X, components, n_features=5):
    # calculate the variance of each feature
    var = np.var(X, axis=0)
    # calculate the variance of each component
    var_comp = np.var(components, axis=0)
    # calculate the contribution of each feature to each component
    contribution = np.abs(np.dot(components.T, X.T) / var_comp[:, None])
    # find the features that contribute most to each component
    features = np.argsort(contribution, axis=0)[::-1, :n_features]
    # print the features that contribute most to each component
    for i in range(contribution.shape[1]):
        print("Component {}: {}".format(i, features[:, i]))


### How to find the features that are contributing the most to the variance along the first principal component?
We can use the following code to find the features that are contributing the most to the variance along the first principal component.
1. We can measure the correlation between the features and the principal components.
```python
def find_features_contributing_most_to_variance_along_first_principal_component(X, components_to_use):
    correlation = np.corrcoef(X.T, components_to_use)
    return np.argsort(correlation[0, :-1])[::-1]
```
2. Variance of the features along the component
```python
def find_features_contributing_most_to_variance_along_first_principal_component(X, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(X)
    first_principal_component = pca.components_[0]
    variance = np.var(X * first_principal_component, axis=0)
    return np.argsort(variance)[::-1]
```

#### Calculate the Loadings
The loadings are the correlation between the features and the principal components. For each principal component, calculate the loading of each variable as the correlation between the variable and the principal component. Specifically, the loading for the i-th variable in the j-th principal component is given by:

$$
loading(i,j) = eigenvector(i,j) * sqrt(eigenvalue(j))
$$

where eigenvector(i,j) is the i-th element of the j-th eigenvector, and eigenvalue(j) is the j-th eigenvalue.


In [65]:
# PCA
xt, U = pca_svd(X, n_components=4, return_components=True)
# find the features that contribute most to each component
find_features_contributing_most_to_variance_to_components(X, U, n_features=5)
 


Explained variance:  [0.90854251 0.9976569  0.99995423 1.        ]
Component 0: [0 1 2 3]
Component 1: [0 1 2 3]
Component 2: [0 1 2 3]
Component 3: [0 1 2 3]
Component 4: [0 1 3 2]


IndexError: index 5 is out of bounds for axis 1 with size 5

In [66]:
X_transformed = np.dot(X, U)
# Calculate the similarity between the original data and the projected data to the first component
similarity = np.corrcoef(X.T, X_transformed[:, 0])
# find the features that contribute most to the first component
features = np.argsort(similarity[0, :-1])[::-1][:5]
print("Features contributing most to the first component: {}".format(features))

# Print the correlation between the original data and the projected data to the first component
print("Correlation between the original data and the projected data to the first component: {}".format(similarity[0, -1]))

Features contributing most to the first component: [0 2 3 1]
Correlation between the original data and the projected data to the first component: -0.8901687648612954


In [71]:
correlation = np.corrcoef(X.T, X_transformed[:, 0])
correlation[np.argsort(correlation[0, :-1])[::-1]]

array([[ 1.        , -0.11756978,  0.87175378,  0.81794113, -0.89016876],
       [ 0.87175378, -0.4284401 ,  1.        ,  0.96286543, -0.99155518],
       [ 0.81794113, -0.36612593,  0.96286543,  1.        , -0.96497896],
       [-0.11756978,  1.        , -0.4284401 , -0.36612593,  0.46014271]])

In [88]:
# calculate pearson correlation between two variables
def pearson_correlation(x, y):
    # calculate mean and standard deviation
    x_mean, y_mean = np.mean(x), np.mean(y)
    x_std, y_std = np.std(x), np.std(y)
    # calculate covariance
    covariance = np.sum((x - x_mean) * (y - y_mean)) / len(x)
    # calculate correlation
    correlation = covariance / (x_std * y_std)
    return correlation

# Print the correlation between the original data and the projected data to each component
for i in range(U.shape[1]):
    X_transformed = np.dot(X, U[:, i])
    corr = [pearson_correlation(X[:, j], X_transformed) for j in range(X.shape[1])]
    print ("Component {}: {}".format(i, corr))

Component 0: [-0.8901687648612947, 0.4601427064479079, -0.9915551834193612, -0.9649789606692488]
Component 1: [-0.36082988811302424, -0.8827162691623837, -0.023415188379165508, -0.06399984704374657]
Component 2: [0.27565766677723386, -0.09361987381838889, -0.054446991873719514, -0.24298265497845473]
Component 3: [0.03760601888781117, -0.017776306845519417, -0.11534978224196103, 0.07535950121713178]


(150,)